In [5]:
import socket
import requests
import dns.resolver
from bs4 import BeautifulSoup
import mysql.connector

#get the IP address
def get_ip(domain):
    try:
        return socket.gethostbyname(domain)
    except socket.gaierror:
        return None
#get the NS records
def get_ns_record(domain):
    try:
        answers = dns.resolver.resolve(domain, 'NS')
        return [rdata.to_text() for rdata in answers]
    except Exception as e:
        return []
#get HTTP status code
def get_http_status_code(domain):
    try:
        response = requests.get(f"http://{domain}", timeout=5)
        return response.status_code, response.text
    except requests.RequestException:
        return None, None
#get the content of the <title> tag
def get_title(html):
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        title_tag = soup.find('title')
        return str(title_tag.string) if title_tag else None
    return None

#get all requested data
def collect_data(domain):
    ip = get_ip(domain)
    ns_record = get_ns_record(domain)
    status_code, html = get_http_status_code(domain)
    title = get_title(html)
    return (domain, ip, ','.join(ns_record), status_code, title)
#create a database and a table to store the collected data
def create_database():
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='domains'
    )
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS domain_data (
                        domain VARCHAR(255) PRIMARY KEY,
                        ip VARCHAR(255),
                        ns_record TEXT,
                        status_code INT,
                        title TEXT
                      )''')
    conn.commit()
    return conn
#store the domain name, IP address, NS records, HTTP status code, and title.
def insert_data(conn, data):
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO domain_data 
                      (domain, ip, ns_record, status_code, title)
                      VALUES (%s, %s, %s, %s, %s)
                      ON DUPLICATE KEY UPDATE
                      ip=VALUES(ip), ns_record=VALUES(ns_record), status_code=VALUES(status_code), title=VALUES(title)''', data)
    conn.commit()

def main(domains):
    conn = create_database()
    for domain in domains:
        data = collect_data(domain)
        insert_data(conn, data)
    conn.close()
#parse the domains file
if __name__ == "__main__":
    with open('List.txt', 'r') as file:
        domains = [line.strip() for line in file.readlines() if line.strip()]
    main(domains)



50 minutes

--Domains with IP ending in an odd number

SELECT domain, ip 
FROM domain_data 
WHERE CAST(SUBSTR(ip, -1) AS INTEGER) % 2 = 1;




--Domains with TLD .com and Name Server *.cloudflare.com


SELECT domain, ns_record 
FROM domain_data 
WHERE domain LIKE '%.com' AND ns_record LIKE '%cloudflare.com%';




--Domains with HTTP status code 200 and "news" in the <title> tag


SELECT domain, status_code, title 
FROM domain_data 
WHERE status_code = 200 AND title LIKE '%news%';


In [6]:
import mysql.connector

def create_connection():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='domains'
    )

def execute_query(query):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    conn.close()
    return results

def print_results(results):
    for row in results:
        print(row)

if __name__ == "__main__":
    # Query 1: Domains with IP ending in an odd number
    query1 = """
    SELECT domain, ip 
    FROM domain_data 
    WHERE CAST(RIGHT(ip, 1) AS UNSIGNED) % 2 = 1;
    """
    results1 = execute_query(query1)
    print("Domains with IP ending in an odd number:")
    print_results(results1)

    # Query 2: Domains with TLD .com and Name Server *.cloudflare.com
    query2 = """
    SELECT domain, ns_record 
    FROM domain_data 
    WHERE domain LIKE '%.com' AND ns_record LIKE '%cloudflare.com%';
    """
    results2 = execute_query(query2)
    print("\nDomains with TLD .com and Name Server *.cloudflare.com:")
    print_results(results2)

    # Query 3: Domains with HTTP status code 200 and "news" in the <title> tag
    query3 = """
    SELECT domain, status_code, title 
    FROM domain_data 
    WHERE status_code = 200 AND title LIKE '%news%';
    """
    results3 = execute_query(query3)
    print("\nDomains with HTTP status code 200 and 'news' in the <title> tag:")
    print_results(results3)


Domains with IP ending in an odd number:
('00news.co.kr', '101.101.208.75')
('10news.com', '108.157.214.11')
('1koreannews.com', '15.197.148.33')
('20minutenews.com', '13.248.213.45')
('373news.com', '61.205.65.89')
('3danews.ir', '89.32.248.197')
('7news1.com', '93.127.163.201')
('abcnews.go.com', '52.85.243.9')
('abruzzonews.eu', '217.182.174.223')
('addalinews.com', '172.67.174.201')
('afaqnews.net', '66.29.137.25')
('africaleadnews.com', '109.234.164.141')
('africatimesnews.com', '74.50.117.113')
('afriqueitnews.com', '35.214.176.129')
('akomnews.com', '121.78.246.169')
('alabama-news.com', '212.224.112.219')
('albaladnews.net', '51.222.248.193')
('albawabhnews.com', '104.21.41.205')
('alexandrianews.org', '188.114.99.225')
('allbanglanewspaper.xyz', '172.67.152.1')
('allthewebnews.com', '15.204.210.49')
('almalnews.com', '188.114.98.225')
('almanacnews.com', '192.0.78.183')
('alpenhornnews.com', '172.234.222.143')
('alraynews.net', '176.9.156.213')
('alwatannews.net', '188.114.99.